In [ ]:
from mesmerize_core import *
import numpy as np
from copy import deepcopy
import pandas as pd
import tifffile
from caiman.motion_correction import high_pass_filter_space
from caiman.summary_images import correlation_pnr

**You will need `fastplotlib` installed for the visualizations**

In [ ]:
from fastplotlib import ImageWidget
from ipywidgets import VBox, IntSlider, Layout

In [ ]:
pd.options.display.max_colwidth = 120

# Paths
These are the only variables you will need to modify in this demo notebook. You will need to set the paths according to your own `caiman_data` dir path

Explanation:

`set_parent_raw_data_path()` - This function from `mesmerize_core` sets the **top level raw data directory**. It is generally the top level directory for your raw experimental data. This allows you to move your experiment directory structure between computers, as long as you keep everything under the parent path the same.

For example,

On Linux based systems if you have your experimental data in the following dir:

`/data/my_name/exp_top_level/....`

You could set `/data/my_name` as the "parent raw data path", and you can then move `exp_top_level/...` between computers.

On windows:

`D:/my_name/exp_top_level/...`

You could set `D:/my_name` as the parent raw data path, and you can then move `exp_top_level/...` between computers.

**Even on windows just use `/`, you do not have to worry about the annoying issue of `\\` and `\` on windows if you use `pathlib` :D**

In [ ]:
# for this demo set this dir as the path to your `caiman_data` dir
set_parent_raw_data_path("/home/kushal/caiman_data/")

### Batch path, this is where caiman outputs will be organized

This can be anywhere, it does not need to be under the parent raw data path.

**We recommend using [pathlib](https://docs.python.org/3/library/pathlib.html) instead of manually managing paths as strings. `pathlib` is just a part of the Python standard library, it makes it much easier to deal with paths and saves a lot of time in the long-run! It also makes your paths compatible across operating systems.**

In [ ]:
batch_path = get_parent_raw_data_path().joinpath("mesmerize-cnmfe/batch.pickle")

# Create a new batch

This creates a new pandas `DataFrame` with the columns that are necessary for mesmerize. In mesmerize we call this the **batch DataFrame**. You can add additional columns relevant to your experiment, but do not modify columns used by mesmerize.

Note that when you create a DataFrame you will need to use `load_batch()` to load it later. You cannot use `create_batch()` to overwrite an existing batch DataFrame

In [ ]:
# create a new batch
df = create_batch(batch_path)
# to load existing batches use `load_batch()`
# df = load_batch(batch_path)

# View the dataframe

It is empty with the appropriate columns for mesmerize

In [ ]:
df

In [ ]:
movie_path = get_parent_raw_data_path().joinpath("example_movies/data_endoscope.tif")

# gSig_filt

A high-pass spatial filter is useful for motion correction of miniscope 1p data, or other data which has large amounts of low frequency background flutuations.

The `gSig_filt` param sets the `sigma` of the gaussian kernel used for filtering. We can use fastplotlib to visualize the effects of this parameter. We want to remove the low frequency spatial information from the image to create better template images for motion correction.

Note that this is different from the `gSig` parameter used in CNMF!

In [ ]:
# get out input movie
# if it is memmapable you can use tifffile.memmap
# for other formats you can try LazyTiff, or any suitable lazy loader
input_movie = tifffile.imread(movie_path)

In [ ]:
# create a slider for gSig_filt
slider_gsig_filt = IntSlider(min=1, max=33, step=2,  description="gSig_filt")

def apply_filter(frame):
    # read slider value
    gSig_filt = (slider_gsig_filt.value, slider_gsig_filt.value)
    
    # apply filter
    return high_pass_filter_space(frame, gSig_filt)

# we can use frame_apply feature of `ImageWidget` to apply 
# the filter before displaying frames
funcs = {
    # data_index: function
    1: apply_filter  # filter shown on right plot, index 1
}

# input movie will be shown on left, filtered on right
iw_gs = ImageWidget(
    data=[input_movie, input_movie.copy()],
    frame_apply=funcs,
    vmin_vmax_sliders=True,
    cmap="gnuplot2"
)

def force_update(*args):
    # kinda hacky but forces the images to update 
    # when the gSig_filt slider is moved
    iw_gs.current_index = iw_gs.current_index
    iw_gs.reset_vmin_vmax()

iw_gs.reset_vmin_vmax()
    
slider_gsig_filt.observe(force_update, "value")

VBox([iw_gs.show(), slider_gsig_filt])

# reset vmin vmax when necessary!!!

In [ ]:
# the filtered images will have much lower min and max
# this resets the vmin vmax sliders on the ImageWidget
iw_gs.reset_vmin_vmax()

# Motion correction parameters

Parameters for all algos have the following structure:

```python
{"main": {... params directly passed to caiman}}
```

In [ ]:
params =\
{
    "main":
    {
        "gSig_filt": (3, 3), # a gSig_filt value that brings out "landmarks" in the movie
        "pw_rigid": True,
        "max_shifts": (5, 5),
        "strides": (48, 48),
        "overlaps": (24, 24),
        "max_deviation_rigid": 3,
        "border_nan": "copy",
    }
}

# Add a "batch item", this is the combination of:
* algorithm to run, `algo`
* input movie to run the algorithm on, `input_movie_path`
* parameters for the specified algorithm, `params`
* a name for you to keep track of things, usually the same as the movie filename, `item_name`

In [ ]:
df.caiman.add_item(
    algo="mcorr",
    input_movie_path=movie_path,
    params=params,
    item_name=movie_path.stem
)

df

## We can now see that there is one item, a.k.a. row or pandas `Series`, in the batch dataframe, we can add another item with the same input movie but with different parameters.

### **When adding batch items with the same `input_movie_path` (i.e. same input movie but different parameters) it is useful to give them the same `item_name`.**

Let's just try one with different `gSig_filt`

In [ ]:
params2 =\
{
    "main":
    {
        "gSig_filt": (1, 1), # a gSig_filt value that brings out "landmarks" in the movie
        "pw_rigid": True,
        "max_shifts": (5, 5),
        "strides": (48, 48),
        "overlaps": (24, 24),
        "max_deviation_rigid": 3,
        "border_nan": "copy",
    }
}

df.caiman.add_item(
    algo="mcorr",
    input_movie_path=movie_path,
    params=params2,
    item_name=movie_path.stem
)

df

## We can see that there are two batch items for the same input movie.

### We can also use a `for` loop to add multiple different parameter variants more efficiently.

In [ ]:
# copy the mcorr_params2 dict to make some changes
new_params = deepcopy(params)

# some variants of max_shifts
# but this can be any params
for shifts in [1, 3, 10]: 
    # deep copy is the safest way to copy dicts
    new_params = deepcopy(new_params)
    
    # assign the "max_shifts"
    new_params["main"]["max_shifts"] = (shifts, shifts)
    
    df.caiman.add_item(
      algo='mcorr',
      item_name=movie_path.stem,
      input_movie_path=movie_path,
      params=new_params
    )

In [ ]:
df

## Now we can see that there are many parameter variants, but it is not easy to see the differences in parameters between the rows that have the same `item_name`.

### We can use the `caiman.get_params_diffs()` extension to see the unique parameters between rows with the same `item_name`

In [ ]:
diffs = df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])
diffs

# Indexing rows and running batch item(s)
#### You can run a single batch item by calling `caiman.run()` on a `Series` (row) of the DataFrame. One way to get the row is integer indexing using `df.iloc[index]`

In [ ]:
# get the first batch item
row = df.iloc[0]

### You can see how the various `pandas.Series` extensions are accessible at the level of dataframe rows.

Move the cursor to the end of the following line and press `Tab` on your keyboard. You can select the `caiman.run()` function and press `Shift + Tab` to see the docstring. You can also instead refer to the API docs. https://mesmerize-core.readthedocs.io/en/latest/api/common.html#mesmerize_core.CaimanSeriesExtensions

Note tab completion doesn't work if you use `df.iloc[i].caiman.<method_name>`, you need to apply the indexer to see the docstring.

In [ ]:
row.caiman.

# Run a single batch item

Run the row that we have selected above, on Linux & Mac it will run in subprocess but on Windows it will run in the local kernel. If using the subprocess backend you can use `run(wait=False)` to not block the current kernel when running.

In [ ]:
# run the first "batch item"
# this will run in a subprocess by default on Linux & Mac
# on windows it will run locally
process = row.caiman.run()

# reload dataframe from disk when done
df = df.caiman.reload_from_disk()

In [ ]:
for i, row in df.iterrows():
    if not i > 0: # skip the first item since we've run it already
        continue
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = df.caiman.reload_from_disk()

# Reload the DataFrame to see the outputs information for the mcorr batch item
### It is necessary to ALWAYS use `df = df.caiman.reload_from_disk()` after running a single batch item or a loop of batch items. You must not add new batch items until you reload it if you have ran items!

In [ ]:
df = df.caiman.reload_from_disk()

## We can see that the `outputs` column has been filled in

In [ ]:
df

# Visualization using `fastplotlib`
You will need `fastplotlib` installed for this, see https://github.com/kushalkolar/fastplotlib

# Get the input movie and mcorr so we can visualize them

Note that you DO NOT need to manually work with file paths. For tiff input files it returns it as a memmaped array (if possible) with lazy loading. It will try to use a mesmerize `LazyArray` if the file cannot be memmaped.

For other file formats you can pass a function that returns a sliceable object, ideally you want to use lazy loading.

In [ ]:
# you can change the index to look at the mcorr results of different batch items
index = 0

# load input movie, this demo movie isn't memmapable and LazyTiff isn't fully read yet so we'll use tifffile.imread
input_movie = df.iloc[index].caiman.get_input_movie(tifffile.imread)

# load mcorr output movie, this loads it as a memmap
mcorr_movie = df.iloc[index].mcorr.get_output()

# Visualize raw & MCorr movie side-by-side

### fastplotlib `ImageWidget` to visualize raw & mcorr movie side by side

`ImageWidget` assumes `"txy"` dimension order by default for 2D movies. You can set other orders using the `dims_order` kwarg

### High pass filtering seems useful again to see movement more easily

In [ ]:
# high pass filter the data to see shifts more easily
filt = lambda x: high_pass_filter_space(x, df.iloc[index]["params"]["main"]["gSig_filt"])

funcs = {
    0: filt,
    1: filt
}

iw = ImageWidget(
    [input_movie, mcorr_movie.astype(np.float32)],
    vmin_vmax_sliders=True, 
    frame_apply=funcs, # the filter func
    names=["input", "mcorr"],
    cmap="gnuplot2",
)

iw.reset_vmin_vmax()

iw.show()

# Frame averaging with a rolling window using `ImageWidget` "window functions".

## This makes it easier to visually inspect motion

In [ ]:
iw.window_funcs = {"t": (np.mean, 11)}

## Close the canvas to free up GPU processing time, not necessary if you have a powerful GPU

In [ ]:
iw.plot.canvas.close()

## With `ImageWidget` you can view all 5 mcorr results simultaneously!

### This depends on your hard drive's capabilities

In [ ]:
# first item is just the raw movie
movies = [df.iloc[0].caiman.get_input_movie(tifffile.imread)]

# subplot titles
subplot_names = ["raw"]

# we will use the mean images later
means = [df.iloc[0].caiman.get_projection("mean")]

# add all the mcorr outputs to the list
for i, row in df.iterrows():
    # add to the list of movies to plot
    movies.append(row.mcorr.get_output())
    
    # subplot title to show dataframe index
    subplot_names.append(f"ix: {i}")

# filter using the same sigma to make visualization easier
# same filter for all movies, this syntax is just dictionary comprehension
filt = {subplot_ix: lambda frame: high_pass_filter_space(frame, (1, 1)) for subplot_ix in range(len(movies))}

# create the widget
mcorr_iw_multiple = ImageWidget(
    data=movies,  # list of movies
    window_funcs={"t": (np.mean, 3)}, # window_funcs is also a kwarg
    vmin_vmax_sliders=True,
    frame_apply=filt, # same func for all
    names=subplot_names,  # subplot names used for titles
    cmap="gnuplot2"
)

mcorr_iw_multiple.reset_vmin_vmax()

mcorr_iw_multiple.show()

# reset vmin vmax if necessary 

In [ ]:
mcorr_iw_multiple.reset_vmin_vmax()

# All the movies here look pretty good so I'll continue with `index = 0`. You can cleanup the DataFrame and remove all other items.

### You can remove batch items (i.e. rows) using `df.caiman.remove_item(<item_uuid>)`

**Note that this also cleans up the output data in the batch directory!**

In [ ]:
# make a list of rows we want to keep using the uuids
rows_keep = [df.iloc[3].uuid]
rows_keep

In [ ]:
for i, row in df.iterrows():
    if row.uuid not in rows_keep:
        df.caiman.remove_item(row.uuid)

df

# CNMF
## corr-pnr seeding

This visualization is to help determine values for `min_corr` (correlation) and `min_pnr` (peak to noise ratio) for seeding CNMFE. Pixels below these thresholds will be excluded from the results.

If `correlation_pnr` takes a long time you can increase the subsample to make it larger than `2`. Example: `mcorr_movie[::5]`

You should try different values of `gSig`, this is different from `gSig_filt`. You will use this gSig as a CNMFE param as well.

In [ ]:
gSig = 3
corr, pnr = correlation_pnr(mcorr_movie[::2], gSig=gSig, swap_dim=False)

In [ ]:
# to show the correlation and pnr images
iw_corr_pnr = ImageWidget(
    [corr, pnr], 
    names=["corr", "pnr"],
    vmin_vmax_sliders=True, 
    cmap="turbo"
)

# some slider stuff
iw_corr_pnr.vmin_vmax_sliders[0].description = "corr"
iw_corr_pnr.vmin_vmax_sliders[1].description = "pnr"

iw_corr_pnr.vmin_vmax_sliders[0].max = corr.max()
iw_corr_pnr.vmin_vmax_sliders[1].max = pnr.max()

# mcorr vids, we will display thresholded mcorr vids
mcorr_vids = [mcorr_movie.astype(np.float32) for i in range(4)]

# sync the threshold image widget with the corr-pnr plot
grid_plot_kwargs = {
    "controllers": [[iw_corr_pnr.plot["corr"].controller]*2]*2
}

iw_thres_movie = ImageWidget(
    mcorr_vids, 
    names=["> corr", "> pnr", "< corr", "< pnr"],
    vmin_vmax_sliders=True,
    # sync this with the corr-pnr plot
    grid_plot_kwargs=grid_plot_kwargs,
    cmap="gnuplot2"
)

# display threshold of the spatially filtered movie
def spatial_filter(frame):
    f = high_pass_filter_space(frame, (3, 3))
    return f


# threshold
def threshold(frame, mask):
    # optionally use spatial filter
    t = spatial_filter(frame)
    
    t = t.copy()
    
    t[mask] = t.min()
    
    return t

# dict of threshold lambda wrappers to set on ImageWidget
# this sets the frame_apply for each subplot
threshold_funcs = {
    0: lambda frame: threshold(frame, corr < iw_corr_pnr.vmin_vmax_sliders[0].value[0]),
    1: lambda frame: threshold(frame, pnr < iw_corr_pnr.vmin_vmax_sliders[1].value[0]),
    2: lambda frame: threshold(frame, corr > iw_corr_pnr.vmin_vmax_sliders[0].value[0]),
    3: lambda frame: threshold(frame, pnr > iw_corr_pnr.vmin_vmax_sliders[1].value[0])
}

# set the dict of lambda wrappers
iw_thres_movie.frame_apply = threshold_funcs

# hacky way to force the threshold plots to update
# when the corr pnr sliders move
def force_update_thresholds_plots(*args):
    iw_thres_movie.current_index = iw_thres_movie.current_index

iw_corr_pnr.vmin_vmax_sliders[0].observe(force_update_thresholds_plots)
iw_corr_pnr.vmin_vmax_sliders[1].observe(force_update_thresholds_plots)

force_update_thresholds_plots()

# iw_thres_movie.reset_vmin_vmax()
    
iw_corr_pnr.plot.canvas.set_logical_size(650, 300)
iw_thres_movie.plot.canvas.set_logical_size(650, 600)

VBox([iw_corr_pnr.show(), iw_thres_movie.show()])

# reset vmin vmax when necessary!!!

In [ ]:
iw_thres_movie.reset_vmin_vmax()

### corr and pnr values from the plot

In [ ]:
corr_pnr = {
    'min_corr': iw_corr_pnr.vmin_vmax_sliders[0].value[0], # corr value from previous plot
    'min_pnr': iw_corr_pnr.vmin_vmax_sliders[1].value[0],  # PNR value from previous plot
}
corr_pnr

In [ ]:
params_cnmfe =\
{
    "main":
    {
        'method_init': 'corr_pnr',  # use this for 1 photon
        'K': None,
        'gSig': (gSig, gSig),
        'gSiz': (4 * gSig + 1, 4 * gSig + 1),
        'merge_thr': 0.7,
        'p': 1,
        'tsub': 2,
        'ssub': 1,
        'rf': 40,
        'stride': 20,
        'only_init': True,    # set it to True to run CNMF-E
        'nb': 0,
        'nb_patch': 0,
        'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
        'low_rank_background': None,
        'update_background_components': True,  # sometimes setting to False improve the results
        'normalize_init': False,               # just leave as is
        'center_psf': True,                    # leave as is for 1 photon
        'ssub_B': 2,
        'ring_size_factor': 1.4,
        'del_duplicates': True,                # whether to remove duplicates from initialization
        **corr_pnr # unpack corr_pnr vals into here
    }
}

### Add a single cnmf item to the batch

When you use `algo="cnmfe"`, it basically forces the following parameters:
```python
"method_init": "corr_pnr",
"n_processes": n_processes,
"only_init": True,  # for 1p
"center_psf": True,  # for 1p
"normalize_init": False,  # for 1p
```

In [ ]:
df.caiman.add_item(
    algo="cnmfe",
    input_movie_path=df.iloc[0],
    params=params_cnmfe,
    item_name=df.iloc[0]["item_name"]
)

df

### Just like with motion correction, we can use loops to add multiple parameter variants. This is useful to perform a parameter search to find the params that work best for your dataset. Here I will use `itertools.product` which is better than deeply nested loops.

In [ ]:
from itertools import product

# variants of several parameters
# you can make lists for as many params as you want
K_variants = [None, 10]
merge_thr_variants = [0.6, 0.8, 0.9, 0.98]

# always use deepcopy like before
new_params_cnmf = deepcopy(params_cnmfe)

# create a parameter grid
# product is a nice way to create all combinations of multiple iterables like lists
parameter_grid = product(K_variants, merge_thr_variants)

# a single for loop to go through all the various parameter combinations
for K, merge_thr in parameter_grid:
    # deep copy params dict just like before
    new_params_cnmf = deepcopy(new_params_cnmf)
    
    # one set of parameter combinations
    new_params_cnmf["main"]["K"] = K
    new_params_cnmf["main"]["merge_thr"] = merge_thr
    
    # add param combination variant to batch
    df.caiman.add_item(
        algo="cnmfe",
        item_name=df.iloc[0]["item_name"],
        input_movie_path=df.iloc[0],
        params=new_params_cnmf
    )

### See that there are a lot of new cnmf batch items

In [ ]:
df

## Since it is difficult to see the different parameter variants above, we can just view the diffs

### The index numbers on the diffs correspond to the indices in the parent DataFrame above

In [ ]:
df.caiman.get_params_diffs(algo="cnmfe", item_name=df.iloc[1]["item_name"])

# Run the added CNMFE items

### First, this is how you can filter a pandas DataFrame using multiple columns. This gives you the rows (batch items) using the "cnmf" `"algo"` and those that match a particular `"item_name"`

In [ ]:
df[
    (df["algo"] == "cnmfe") &  # algo
    (df["item_name"] == df.iloc[0]["item_name"])  # item name
]

In [ ]:
for i, row in df[
    (df["algo"] == "cnmfe") &
    (df["item_name"] == df.iloc[0]["item_name"])
].iterrows():
    
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = load_batch(df.paths.get_batch_path())

### We now have CNMF outputs

In [ ]:
df = df.caiman.reload_from_disk()
df[df["algo"] == "cnmfe"]